In [ ]:
import numpy as np
import math
from functools import reduce
import time
import sys
import os
import numpy as np

path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)
from pathlib import Path
import nidaqmx

def lcm(a, b):
    """Calculate the least common multiple of two numbers."""
    return abs(a * b) // math.gcd(a, b)


def lcm_of_list(numbers):
    """Find the LCM of a list of numbers."""
    return reduce(lcm, numbers)

def seqtime(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])

# some constants
Hz = 1e-9 # GHz
kHz = 1e-6 # GHz
MHz = 1e-3 # GHz
pi = np.pi


In [ ]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.stream_readers import AnalogSingleChannelReader

from hardware import config as hcf
from hardware.hardwaremanager import HardwareManager
from hardware.pulser.pulser import (
    OutputState,
    TriggerStart,
    TriggerRearm,
    HIGH,
    LOW,
    INF,
    REPEAT_INFINITELY
)
timebase = lcm_of_list(
    [hcf.VDISYN_timebase, hcf.SIDIG_timebase, hcf.PS_timebase, hcf.RSRF_timebase]
)

hm = HardwareManager()
hm.add_default_hardware()

# Some Parameters 
the parameters that will be used in the measurement and need to be set in the GUI


In [ ]:
# ---------------------
# Settings for MW
mw_freq = 398.54667  # GHz
mw_hopspan = 2*3.5E-3 # GHz
# mw_hoptime = 1000.0 # ns

mw_powervolt = 5.0  # voltage 0.0 to 5.0
mw_phasevolt = 0.0  # voltage 0.0 to 5.0

# setting for data acquisition
min_volt = -10.0E-3 # [V]
max_volt = 150.0E-3

# setting for laser
laser_current = 95.0  # percentage

# settings for the RF
rf_freq = 600 # MHz

# -------------------
# setting for the sensing sequence
# # nuclear spin preparation
ninit_pihalf = 1000 #ns

# in a locking block
t_lock_idle = 20 # ns
t_lock_rf = 3980 # ns
t_lock = t_lock_idle + t_lock_rf
n_lock = 2
t_lbloc = t_lock*n_lock

t_lbloc_mwwait = 20.0
t_lbloc_mw = t_lbloc-t_lbloc_mwwait


# in a laser read and init block
t_ribloc = t_lbloc
t_ribloc_wait = 400.0
t_ribloc_isc = 800.0
t_ribloc_laser = t_ribloc - t_ribloc_wait - t_ribloc_isc

# in free evolution
t_fevol = 80e3 # [ns] it is better to be in the multiple of lblock time
n_fevol_div = t_fevol // t_lbloc
t_fevol = n_fevol_div * t_lbloc

# dark reference block
t_drbloc = t_ribloc + t_lbloc
# bright reference block
t_brbloc = t_ribloc

# clearing block using a train of laser pulses, clean the memory effect
t_cbloc = t_fevol - t_brbloc - t_drbloc - t_ribloc
t_cbloc_laser = 10 # ns
t_cbloc_isc = 400 #ns
n_cbloc_laser = int(t_cbloc//(t_cbloc_laser+t_cbloc_isc)//4)
t_cbloc_pad = t_cbloc - n_cbloc_laser*(t_cbloc_laser+t_cbloc_isc)

n_track = 5000 # repetition of the free evolution + the detection block 
t_atrack = t_lbloc + t_fevol
T_alltrack = n_track*t_atrack # [ns] total time = n_track*(free evolution time + detection block time * number of lbloc)

# time_stop = 3600.0*6
time_stop = 600.0
rate_refresh = 10 # 10Hz refreshing rate
# -------------------
paraset = dict(
    mw_freq = mw_freq,
    # mw_hoptime=mw_hoptime,
    mw_hopspan=mw_hopspan,
    mw_powervolt=mw_powervolt,
    mw_phasevolt=mw_phasevolt,
    min_volt=min_volt,
    max_volt=max_volt,
    laser_current=laser_current,
    rf_freq=rf_freq,
    # pulse sequence------
    ninit_pihalf=ninit_pihalf,
    t_lock_idle = t_lock_idle, 
    t_lock_rf = t_lock_rf, 
    n_lock = n_lock, 
    #--------------
    t_lbloc_mwwait = t_lbloc_mwwait,
    t_lbloc_mw = t_lbloc_mw, 
    t_ribloc = t_ribloc, 
    t_ribloc_wait = t_ribloc_wait,
    t_ribloc_isc = t_ribloc_isc,
    t_ribloc_laser = t_ribloc_laser,
    #---------
    t_fevol=t_fevol,
    t_brbloc=t_brbloc,
    t_drbloc=t_drbloc,
    #-------------
    t_cbloc = t_cbloc,
    t_cbloc_laser = t_cbloc_laser,
    t_cbloc_isc = t_cbloc_isc,
    n_cbloc_laser = n_cbloc_laser,
    t_cbloc_pad = t_cbloc_pad,
    n_track=n_track
)

In [ ]:
# parameter checking

def check_parameters(paraset):
    assert paraset["mw_powervolt"] <= 5 and paraset["mw_powervolt"]>=0
    assert paraset["mw_phasevolt"] <= 5 and paraset["mw_phasevolt"]>=0
    timeparameters = [
        paraset["t_lbloc_mw"], 
        paraset["t_lbloc_mwwait"],
        paraset["t_lock_idle"],
        paraset["t_lock_rf"],
        paraset["ninit_pihalf"],
        paraset["t_fevol"]
    ]
    for tt in timeparameters:
        assert tt%timebase == 0
    t_lbloc = paraset["n_lock"]*(paraset["t_lock_idle"]+paraset["t_lock_rf"])
    assert paraset["t_fevol"]%t_lbloc == 0
    assert paraset["ninit_pihalf"]*2 <= t_lbloc
    assert paraset["t_fevol"] >= t_lbloc+paraset["t_drbloc"]+paraset["t_brbloc"]+paraset["t_ribloc"]
    assert t_cbloc == paraset["t_cbloc_pad"]+(paraset["t_cbloc_laser"]+paraset["t_cbloc_isc"])*paraset["n_cbloc_laser"]
check_parameters(paraset)

## Pulse Sequence for single-point frequency tracking of ODMR with emulated nuclear radio signals

In [ ]:
def seqtime_tb(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])
def seqtime_cb(seq_cb):
    return np.sum([pulse[-0] for pulse in seq_cb])

In [ ]:
# --------------------------------------------------------------------------------
# set up the pulse sequence ----------------------------------------------------------
# set the channel general offsets--------------------------------------------------
ps_choffs = {
    "laser": 0,
    "dclk": 0.0,
    "dtrig": 0.0,
    "mwA": 0,
    "mwB": 0,
    "rftrig": 0,
    "Bz": 0,  # AO 0
    "Bx": 0,  # AO 1
}
hm.pg.setChOffset(ps_choffs)
# ------------------------------------------------------------------------------
hm.pg.resetSeq()


t_lbloc = paraset["n_lock"]*(paraset["t_lock_idle"]+paraset["t_lock_rf"])
t_rf_pihalf = paraset["ninit_pihalf"]
n_track = paraset["n_track"]
n_atrack_div = int(t_atrack//t_lbloc)
subseq_ninit = [
    (["rftrig","dtrig"], t_lbloc)
]
t_fevol = paraset["t_fevol"] 
t_atrack = t_fevol+t_lbloc
t_trig = 200
t_ninit = t_lbloc + t_ribloc
t_readat = paraset["t_ribloc_wait"]+900+160

subseq_fevol = [([], t_fevol)]
subseq_ninit_rf = [(t_ninit-t_rf_pihalf-t_ribloc, LOW), (t_rf_pihalf, HIGH), (t_ribloc, LOW)]
subseq_ninit_dtrig = [(t_trig, HIGH), (t_ninit-t_trig, LOW)] 
subseq_ninit_pad = [(t_ninit, LOW)]
n_fevol_div = int(paraset["t_fevol"]//t_lbloc)
n_fevol_div_half = n_fevol_div//2

subseq_cbloc_laser =  [(t_cbloc_laser, HIGH), (t_cbloc_isc, LOW)]*n_cbloc_laser+[(t_cbloc_pad, LOW)]
# subseq_cbloc_laser = [(t_cbloc_laser, HIGH), (t_cbloc_isc, LOW)]*n_cbloc_laser+[(t_cbloc_pad, LOW)]
subseq_ribloc_laser = [(t_ribloc_wait, LOW), (t_ribloc_laser, HIGH), (t_ribloc_isc, LOW)]
subseq_drbloc_laser = [(t_drbloc-t_ribloc, LOW)] + subseq_ribloc_laser
subseq_brbloc_laser = subseq_ribloc_laser

subseq_drbloc_mw = [(t_lbloc_mwwait, LOW), (t_lbloc_mw, HIGH)] + [(t_ribloc, LOW)]
subseq_brbloc_mw = [t_brbloc, LOW]
subseq_cbloc_mw = [(t_cbloc, LOW)]
t_lblochalf = int(t_lbloc/2//timebase)*timebase

seq_laser = subseq_ninit_pad+n_track*(subseq_cbloc_laser+subseq_drbloc_laser+subseq_brbloc_laser+[(t_lbloc, LOW)]+subseq_ribloc_laser)
seq_mwA = subseq_ninit_pad+n_track*([(t_cbloc, LOW)]+subseq_drbloc_mw+[(t_brbloc, LOW)]+[(t_lbloc_mwwait, LOW), (t_lbloc_mw, HIGH)]+[(t_ribloc, LOW)])
# seq_mwA = subseq_init_pad+(subseq_fevol+[(t_lbloc, LOW)]*n_lbloc)*n_track
# seq_dclk = subseq_init_pad + [(t_lbloc-t_lblochalf, LOW), (t_trig, HIGH), (t_lblochalf-t_trig, LOW)]*(n_lbloc+n_fevol_div)*n_track
seq_dclk = subseq_ninit_pad + n_track*[(t_readat, LOW), (t_trig, HIGH),(t_ribloc-t_readat-t_trig, LOW)]*n_atrack_div
seq_dtrig = subseq_ninit_dtrig + [(t_atrack*n_track, LOW)]
seq_rftrig = subseq_ninit_rf + [(t_atrack*n_track, LOW)]


hm.pg.setDigital("laser", seq_laser, offset=True)
hm.pg.setDigital("mwA", seq_mwA, offset=True)
# seq_mwB = seq_mwA
# hm.pg.setDigital("mwB", seq_mwB, offset=True)
hm.pg.setDigital("dclk", seq_dclk, offset=True)
hm.pg.setDigital("dtrig", seq_dtrig, offset=True)
hm.pg.setDigital("rftrig", seq_rftrig, offset=True)


### Fake Nuclear Signal
The emulated signals are based on the assumption that the nuclear driving is much faster than the preccession of the nuclear spins in the rotating frame, such that the magnetic signal align z direction in each detection block is modulated by $\cos(\omega_n t_i)$ where $\omega_n$ is the detuned nuclear Larmor frequency and $t_i$ is the time at the begining of $i$-th detection block. 

$$b_i(t) = b_0 sin(\Omega_{rf} \tau)\cos(\omega t_i+\phi_{noise})$$
where $\tau=t-t_i$ starts from the RF drive in each detection block

In [ ]:
# to generate fake signal for testing
amp_a = 0.0
amp_b = 1.0
amp_c = 0.0
omega = 2 * pi * 15 * Hz
omega_b = 2 * pi * 32.425266 * Hz
omega_c = 2 * pi * 25 * Hz
decay_a = 0.2*Hz
decay_b = 2*Hz
decay_c = 0.2*Hz
rfomega = 2 * pi * 80 * kHz
dt_inlock = 199.0
t_lock_idle = paraset["t_lock_idle"]
t_lock_rf = paraset["t_lock_rf"]
assert t_lock_rf%dt_inlock==0

tarray_inlock = np.arange(0, t_lock_rf
                        , dt_inlock)
bzlevel0 = np.sin(rfomega*tarray_inlock)
bzlevel0_flip = np.flip(bzlevel0)
i_track = 0
wf_Bz_xpc = []
wf_Bz_xpc += [(t_ninit-t_ribloc, LOW)]
wf_Bz_ypc = []
wf_Bz_ypc += subseq_ninit_pad
# randphase_a = np.random.random(1)*pi
# randphase_b = np.random.random(1)*pi
randphase_a = 0.0
randphase_b = 0.0
randphase_c = 0.0
bzevol_cos = np.zeros(n_track)
for it in range(n_track):
    t_evol = t_ninit + t_atrack*it + t_fevol + t_lock_idle
    factor_evol_cos = float(amp_a*np.cos(omega*t_evol+randphase_a)*np.exp(-decay_a*t_evol)+\
                            amp_b*np.cos(omega_b*t_evol+randphase_b)*np.exp(-decay_b*t_evol)+\
                            amp_c*np.cos(omega_c*t_evol+randphase_c)*np.exp(-decay_c*t_evol))
                            
    factor_evol_cos /= (amp_a+amp_b+amp_c)
    # factor_evol_cos *= 0.1
    bzevol_cos[it] = factor_evol_cos
    # factor_evol_sin = amp_a*np.sin(omega*(t_evol+t_lbloc+t_lbloc_idle)+randphase_a)+amp_b*np.sin(omega_b*(t_evol+t_lbloc+t_lbloc_idle)+randphase_b)
    # factor_evol_sin /= (amp_a+amp_b)
    bzlevel_xpc = bzlevel0*factor_evol_cos # X phase cycle
    bzlevel_xpc_flip = bzlevel0_flip*factor_evol_cos
    # bzlevel_ypc = bzlevel0*factor_evol_sin # Y phase cycle

    # comment out if trying smooth radio signal---------
    # bzlevel_ypc_flip = bzlevel0_flip*factor_evol_sin
    wf_bz_inlock_xpc = [(t_lock_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_xpc]
    wf_bz_inlock_xpc +=  [(t_lock_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_xpc_flip]
    # wf_bz_inlock_ypc = [(t_lbloc_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_ypc]
    # wf_bz_inlock_ypc +=  [(t_lbloc_idle, 0)]+[(dt_inlock, bzl) for bzl in bzlevel_ypc_flip]
    # # wf_bz_inlock = [(t_lbloc, 1)]
    wf_Bz_xpc += [(t_fevol, 0)]+wf_bz_inlock_xpc
    # # wf_Bz_ypc += [(t_fevol, 0)]+wf_bz_inlock_ypc
    # ----------------------------------------------------------
    # wf_Bz_xpc += [(t_atrack, factor_evol_cos)]
    # wf_Bz_ypc += [(t_fevol, 0)]+wf_bz_inlock_ypc
wf_Bz_xpc += [(t_ribloc, LOW)]

hm.pg.setAnalog("Bz", wf_Bz_xpc, offset=True)
# hm.pg.setAnalog("Bx", wf_Bz_xpc, offset=True)

In [ ]:
assert seqtime_cb(seq_laser) == seqtime_cb(seq_mwA) == seqtime_cb(wf_Bz_xpc) == seqtime_cb(seq_dclk)

In [ ]:
hm.pg.plotSeq(plot_all=False)

In [ ]:
hm.pg.setTrigger(TriggerStart.SOFTWARE, rearm=TriggerRearm.AUTO)
hm.pg.stream(n_runs=REPEAT_INFINITELY)

In [ ]:

# set up the data acquisition with NI daq
# signal reading parameters
min_volt = paraset["min_volt"]
max_volt = paraset["max_volt"]
rate_sample = 1.0/t_lbloc/Hz # sampling rate of the NI daq, Hz
num_read_inatrack = n_atrack_div
num_read_refresh = rate_sample/rate_refresh
n_track_refresh = int(num_read_refresh//num_read_inatrack)
num_read_refresh = int(n_track_refresh*num_read_inatrack)

num_readsample = n_track*n_atrack_div

timeout_read = max(2*num_read_refresh/rate_sample, 10)
# buffer_size = num_readsample
# buffer_temp = np.zeros(buffer_size, dtype=np.float64, order='C')
buffer_refresh = np.zeros(num_read_refresh, dtype=np.float64, order='C')
n_dataslot = 4# dark ref, bright ref, nolaser, sig
buffer_data = np.zeros((n_track, n_dataslot), dtype=np.float64, order='C') 


In [ ]:
readtask = nidaqmx.Task("readsignal")
# readtask.close()
readtask.ai_channels.add_ai_voltage_chan(
            hcf.NI_ch_APD,"",
            TerminalConfiguration.DIFF,
            min_volt,max_volt,
            VoltageUnits.VOLTS
        )
# readtask.timing.cfg_samp_clk_timing(samplerate_read, source="", active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=num_readsample)
readtask.timing.cfg_samp_clk_timing(
    rate_sample, 
    source=hcf.NI_ch_Clock, 
    active_edge=Edge.RISING, 
    sample_mode=AcquisitionType.CONTINUOUS)
read_trig = readtask.triggers.start_trigger
read_trig.cfg_dig_edge_start_trig(hcf.NI_ch_Trig, Edge.RISING)

reader = AnalogSingleChannelReader(readtask.in_stream)
reader.read_all_avail_samp  = True

In [ ]:
# -----------------------------------------------------------------------
# set the MW frequency --------------------------------------------------
try:
    hm.mwsyn.open()
except Exception as ee:
    print(ee)
mw_freq = paraset["mw_freq"] + paraset["mw_hopspan"]/2.0
errorbyte, freq_actual = hm.mwsyn.cw_frequency(mw_freq / hcf.VDISYN_multiplier)
print(f"CW Freqeuncy Setting Sent:{mw_freq/hcf.VDISYN_multiplier} GHz")
print(f"Actual Output CW Freqeuncy :{freq_actual} GHz")

In [ ]:
# -----------------------------------------------------------------------
# set MW power and phase using NIIO ------------------------------------
mwpower_vlevel = paraset["mw_powervolt"]  # 5V equals to max power
task_uca = nidaqmx.Task("UCA")  # user controlled attenuation
task_uca.ao_channels.add_ao_voltage_chan(hcf.NI_ch_UCA, min_val=0, max_val=10)
# task_uca.timing.cfg_samp_clk_timing(hcf.NI_sampling_max/100.0, sample_mode=AcquisitionType.CONTINUOUS)
task_uca.start()
task_uca.write([mwpower_vlevel], auto_start=False)

mwphase_vlevel = paraset["mw_phasevolt"]  # voltage to phase shifter
task_mwbp = nidaqmx.Task("MW B Phase")  # user controlled attenuation
task_mwbp.ao_channels.add_ao_voltage_chan(hcf.NI_ch_MWBP, min_val=0, max_val=10)
# task_uca.timing.cfg_samp_clk_timing(hcf.NI_sampling_max/100.0, sample_mode=AcquisitionType.CONTINUOUS)
task_mwbp.start()
task_mwbp.write([mwphase_vlevel], auto_start=False)

In [ ]:
# -----------------------------------------------------------------------
# set up laser----------------------------------------------------------
current_percent = paraset["laser_current"]
hm.laser.laser_off()
hm.laser.set_analog_control_mode("current")
hm.laser.set_modulation_state("Pulsed")
hm.laser.set_diode_current(current_percent, save_memory=False)

In [ ]:
hm.laser.laser_on() # turn on laser
readtask.start() # ready to read data
hm.pg.startNow()
start_time = time.time()
idx_run = 0
idx_pointer = 0
num_track_readed = 0
num_repeated = np.zeros(n_track)
while time.time() - start_time < time_stop:
    idx_pointer = idx_pointer%n_track
    num_read = reader.read_many_sample(buffer_refresh, num_read_refresh, timeout_read)
    assert num_read==num_read_refresh
    buffer_refresh_reshape = np.reshape(buffer_refresh, (n_track_refresh, num_read_inatrack))
    if n_track-idx_pointer >= n_track_refresh:
        buffer_data[idx_pointer:idx_pointer+n_track_refresh] += np.copy(buffer_refresh_reshape[:, -(n_dataslot):])
        num_repeated[idx_pointer:idx_pointer+n_track_refresh] += 1
    else:
        # print("unwrapp raw data----")
        n_remaining = n_track-idx_pointer
        n_front = n_track_refresh-n_remaining
        buffer_data[idx_pointer:idx_pointer+n_remaining] += np.copy(buffer_refresh_reshape[:n_remaining, -(n_dataslot):])
        num_repeated[idx_pointer:idx_pointer+n_remaining] += 1
        buffer_data[:n_front] += np.copy(buffer_refresh_reshape[n_remaining:, -(n_dataslot):])
        num_repeated[:n_front] += 1
    idx_pointer += n_track_refresh
    idx_run += 1
    # print(f"Index Pointer: {idx_pointer}, Portion in all tracks: {idx_pointer/n_track}")
for ii in range(n_dataslot):
    buffer_data[:, ii] = np.divide(buffer_data[:, ii], num_repeated)

In [ ]:
hm.pg.forceFinal()
hm.pg.constant(OutputState.ZERO())
hm.pg.reset()
hm.laser.laser_off()
task_mwbp.close()
task_uca.close()
readtask.close()
hm.mwsyn.close_gracefully()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming 't_atrack', 'n_track', and 'buffer_data' are defined
timetime = t_atrack * np.arange(0, n_track, 1) / 1E9

# Create subplots with two columns
fig, axes = plt.subplots(2, 1, figsize=(8, 6))
name = ["dark", "bright", "bg", "signal"]
# Plot the original data (time domain)
# for ii in range(0, n_dataslot):
for ii in [0,1,3]:
    sig_diff = buffer_data[:, ii]-buffer_data[:, 2]
    # sig_diff = buffer_data[:, ii]
    axes[0].plot(timetime, sig_diff*1E3, label=name[ii])
axes[0].set_xlabel("Time [s]")
axes[0].set_ylabel("PL [mV]")
axes[0].set_title("Original Signal")
axes[0].legend()
axes[0].grid()
sig_fft_0 =  np.abs(np.fft.fft(buffer_data[:, 0]))
# Plot the FFT (frequency domain, positive frequencies only)
for ii in range(0, n_dataslot):
# for ii in [0, 1, 3]:
    sig_diff = buffer_data[:, ii] 
    freq = np.fft.fftfreq(len(sig_diff), t_atrack / 1E9)
    sig_fft = np.abs(np.fft.fft(sig_diff))
    # sig_fft = sig_fft - sig_fft_0

    # Get only positive frequencies
    positive_freqs = freq > 0
    axes[1].plot(freq[positive_freqs], sig_fft[positive_freqs], label=f"FFT {name[ii]}")
axes[1].set_xlabel("Frequency [Hz]")
axes[1].set_ylabel("Amplitude")
axes[1].set_title("FFT of Signals (Positive Frequencies)")
axes[1].legend()
axes[1].grid()

# Adjust layout and show the plots
plt.tight_layout()
plt.show()


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Assuming 't_atrack', 'n_track', 'buffer_data', and 'n_lbloc' are defined
timetime = t_atrack * np.arange(0, n_track, 1) / 1E9

# Create a Plotly figure
fig = go.Figure()

# # Plot the original data (time domain)
# for ii in range(0, n_dataslot):
#     sig_diff = buffer_data[:, ii] - buffer_data[:, 0]
#     fig.add_trace(go.Scatter(
#         x=timetime,
#         y=sig_diff,
#         mode='lines',
#         name=f"Signal {ii+1} (Time Domain)"
#     ))

# Plot the FFT (frequency domain, positive frequencies only)
# sig_fft_0 = np.abs(np.fft.fft(buffer_data[:, 0]-buffer_data[:, 2]))
sig_fft_0 = np.abs(np.fft.rfft(buffer_data[:, 0]-np.mean(buffer_data[:, 0])))
sig_fft_1 = np.abs(np.fft.rfft(buffer_data[:, 1]-np.mean(buffer_data[:, 1])))

# for ii in range(0, n_dataslot):
for ii in [0,1,3]:
    sig_diff = buffer_data[:, ii]
    # sig_diff = buffer_data[:, ii]
    sig_diff = sig_diff-np.mean(sig_diff)
    freq = np.fft.rfftfreq(len(sig_diff), t_atrack / 1E9)
    sig_fft = np.abs(np.fft.rfft(sig_diff))
    sig_fft = sig_fft-(sig_fft_0+sig_fft_1)/2
    sig_fft = sig_fft/(len(sig_diff)/2)
    # # Get only positive frequencies
    fig.add_trace(go.Scatter(
        x=freq,
        y=sig_fft,
        mode='lines',
        name=f"FFT {name[ii]}"
    ))

# Update layout for better readability
fig.update_layout(
    title="Original Signal and FFT",
    xaxis_title="Time [s] / Frequency [Hz]",
    yaxis_title="Amplitude / PL [V]",
    legend_title="Signal Legend",
    grid=dict(rows=2, columns=1, pattern="independent"),
    height=600,
    width=800
)

# Show the Plotly figure
fig.show()


In [ ]:
import pickle
# Define the file name
fname = f"output/data_dev_pulsed_odmr_tracking_{time.time()}.pkl"

# Assuming 'buffer_data' and 'paraset' are defined
data_to_save = {
    "buffer_data": buffer_data,
    "paraset": paraset
}

# Save the data to a pickle file
with open(fname, "wb") as file:
    pickle.dump(data_to_save, file)

print(f"Data successfully saved to {fname}")


In [ ]:

# fname = "output/data_dev_pulsed_odmr_tracking_test_combBz_allon_lfdecay32long_changedapd_1737710114.0613265.pkl"
# # fname = "output/data_dev_pulsed_odmr_tracking_test_combBz_allon_lfdecay32long_changedapd_1737688388.4160972.pkl"

# with open(fname, "rb") as file:
#     loaded_data = pickle.load(file)

# buffer_data = loaded_data["buffer_data"]
# paraset = loaded_data["paraset"]

In [ ]:
# def music(signal, sampling_rate, num_sources, n_fft=1024):
#     # Compute covariance matrix
#     L = len(signal) // 2
#     X = np.array([signal[i:i+L] for i in range(len(signal) - L + 1)]).T
#     R = np.dot(X, X.T.conj()) / X.shape[1]
    
#     # Eigen decomposition
#     eigvals, eigvecs = np.linalg.eigh(R)
#     noise_subspace = eigvecs[:, :-num_sources]  # Noise eigenvectors
    
#     # MUSIC spectrum
#     freqs = np.linspace(0, sampling_rate/2, n_fft)
#     spectrum = []
#     for f in freqs:
#         steering_vector = np.exp(-1j * 2 * np.pi * f / sampling_rate * np.arange(L))
#         projection = np.sum(np.abs(noise_subspace.T.conj() @ steering_vector[:, None])**2)
#         spectrum.append(1 / projection)
    
#     spectrum = np.array(spectrum)
#     return freqs, spectrum

# # Example: Simulated signal
# fs = 1/t_atrack*1E9
# signal = buffer_data[:, 3]-buffer_data[:, 0]
# signal = signal - np.mean(signal)
# # Estimate frequencies using MUSIC
# num_sources = 1  # Number of signal components
# freqs, spectrum = music(signal, fs, num_sources)
# # Normalize the spectrum
# normalized_spectrum = spectrum / np.max(spectrum)

# # Create the plot
# fig = go.Figure()

# fig.add_trace(go.Scatter(
#     x=freqs,
#     y=normalized_spectrum,
#     mode='lines',
#     name='MUSIC Spectrum'
# ))

# fig.show()



# Simulation TBC



In [ ]:
# def rotation_matrix(axis, theta):
#     """
#     Compute the 3D rotation matrix for a given axis and angle.

#     Parameters:
#         axis (array-like): A unit vector representing the axis of rotation (n_x, n_y, n_z).
#         theta (float): Rotation angle in radians.

#     Returns:
#         numpy.ndarray: A 3x3 rotation matrix.
#     """
#     axis = np.array(axis, dtype=float)
#     axis = axis / np.linalg.norm(axis)  # Ensure the axis is a unit vector
    
#     n_x, n_y, n_z = axis
#     cos_theta = np.cos(theta)
#     sin_theta = np.sin(theta)
#     I = np.eye(3)  # Identity matrix
    
#     # Skew-symmetric cross-product matrix
#     n_cross = np.array([
#         [0, -n_z, n_y],
#         [n_z, 0, -n_x],
#         [-n_y, n_x, 0]
#     ])
    
#     # Rodrigues' rotation formula
#     R = I + sin_theta * n_cross + (1 - cos_theta) * np.dot(n_cross, n_cross)
#     return R

# def spin_state(t, b0, omega, rfomega, t_fe, ):
#     # output the spin state in the rotating frame in terms of S vector
#     # assume at t=0, the spin is initiated to the xy plane
#     randphase = np.random.random(1)*pi
#     xstate = [1, 0, 0]
#     spinstate = rotation_matrix([0,0,1], randphase)*xstate
#     omega_eff = np.sqrt(omega**2 + rfomega**2)
#     theta = omega_eff*t
#     rotation_matrix([0,0,1], randphase)
#     spinstate_trace = []